<a href="https://colab.research.google.com/github/soumabkargit/intro-to-machine-learning/blob/master/LinearRegressionTensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# import packages
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import boston_housing
from tensorflow.keras import Model
from sklearn.preprocessing import StandardScaler

# load dataset and split in train and test sets
(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

# standardize the dataset
scaler_X_train = StandardScaler().fit(X_train)
scaler_X_test = StandardScaler().fit(X_test)
X_train = scaler_X_train.transform(X_train)
X_test = scaler_X_test.transform(X_test)

# reshape y-data to become column vector
y_train = np.reshape(y_train, [-1, 1])
y_test = np.reshape(y_test, [-1, 1])

# build the linear model
class LinearRegressionModel(Model):
  def __init__(self):
    super(LinearRegressionModel, self).__init__()
    # initialize weight and bias variables
    self.weight = tf.Variable(
        initial_value = tf. random.normal(
            [13, 1], dtype=tf.float64),
        trainable=True)
    self.bias = tf.Variable(initial_value = tf.constant(
        1.0, shape=[], dtype=tf.float64), trainable=True)

  def call(self, inputs):
    return tf.add(tf.matmul(inputs, self.weight), self.bias)

model = LinearRegressionModel()

# parameters
batch_size = 32
learning_rate = 0.01

# use tf.data to batch and shuffle the dataset
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(X_train)).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size)

loss_object = tf.keras.losses.MeanSquaredError()

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_rmse = tf.keras.metrics.RootMeanSquaredError(name='train_rmse')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_rmse = tf.keras.metrics.RootMeanSquaredError(name='test_rmse')

# use tf.GradientTape to train the model
@tf.function
def train_step(inputs, labels):
  with tf.GradientTape() as tape:
    predictions = model(inputs)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_rmse(labels, predictions)


@tf.function
def test_step(inputs, labels):
  predictions = model(inputs)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_rmse(labels, predictions)


num_epochs = 1000

for epoch in range(num_epochs):
  for train_inputs, train_labels in train_ds:
    train_step(train_inputs, train_labels)

  for test_inputs, test_labels in test_ds:
    test_step(test_inputs, test_labels)

  template = 'Epoch {}, Loss: {}, RMSE: {}, Test Loss: {}, Test RMSE: {}'

  if ((epoch+1) % 100 == 0):
    print (template.format(epoch+1,
                           train_loss.result(),
                           train_rmse.result(),
                           test_loss.result(),
                           test_rmse.result()))


Epoch 100, Loss: 33.2088508605957, RMSE: 5.77750301361084, Test Loss: 38.435089111328125, Test RMSE: 5.365653991699219
Epoch 200, Loss: 27.709814071655273, RMSE: 5.274987697601318, Test Loss: 33.93333435058594, Test RMSE: 4.978856563568115
Epoch 300, Loss: 25.875425338745117, RMSE: 5.096312999725342, Test Loss: 32.43899154663086, Test RMSE: 4.8442840576171875
Epoch 400, Loss: 24.958229064941406, RMSE: 5.004578113555908, Test Loss: 31.693405151367188, Test RMSE: 4.77574348449707
Epoch 500, Loss: 24.407699584960938, RMSE: 4.948701858520508, Test Loss: 31.246257781982422, Test RMSE: 4.734169960021973
Epoch 600, Loss: 24.04067039489746, RMSE: 4.911094665527344, Test Loss: 30.948335647583008, Test RMSE: 4.706249713897705
Epoch 700, Loss: 23.778507232666016, RMSE: 4.884055137634277, Test Loss: 30.735572814941406, Test RMSE: 4.686212062835693
Epoch 800, Loss: 23.581886291503906, RMSE: 4.86367654800415, Test Loss: 30.575998306274414, Test RMSE: 4.67113733291626
Epoch 900, Loss: 23.429126739501